In [ ]:
import requests
import pandas as pd
import time
from delta_rest_client import DeltaRestClient, OrderType

API_KEY = "EgpjAIduee0zvVKZALtAYUqaWAnl5f"
API_SECRET = "qnV4NyswZQWkZifvRvwQKm1AAveR8fto3MOPGyHBTkFooCphUEeocKTleTGZ"
BASE_URL = "https://api.india.delta.exchange"
ADJUSTMENT_THRESHOLD = 1.3
DELTA_TARGET = 0.18
BROKERAGE_PER_TRADE = 0.5

def safe_get(url, max_retries=5, retry_delay=10):
    for attempt in range(max_retries):
        try:
            res = requests.get(url)
            res.raise_for_status()
            return res
        except requests.exceptions.RequestException as e:
            print(f"Network error: {e}. Retrying in {retry_delay}s... (Attempt {attempt+1}/{max_retries})")
            time.sleep(retry_delay)
    raise Exception(f"Failed to GET {url} after {max_retries} retries.")

delta = DeltaRestClient(
    base_url=BASE_URL,
    api_key=API_KEY,
    api_secret=API_SECRET
)

def get_products():
    url = BASE_URL + "/v2/products"
    res = safe_get(url)
    return res.json().get("result", [])

def get_live_option_tickers(symbol, expiry_str):
    url = (
        BASE_URL +
        f"/v2/tickers?contract_types=call_options,put_options"
        f"&underlying_asset_symbols={symbol}&expiry_date={expiry_str}"
    )
    res = safe_get(url)
    return res.json().get("result", [])

def get_spot_price():
    url = BASE_URL + "/v2/tickers"
    res = safe_get(url)
    tickers = res.json().get("result", [])
    spot = [t for t in tickers if t.get('symbol', '').upper() == 'BTCUSD']
    if spot:
        return float(spot[0]['spot_price'])
    else:
        return None

def place_market_order(delta, product_id, size, side):
    return delta.place_order(
        product_id=product_id,
        size=size,
        side=side,
        order_type=OrderType.MARKET
    )

def estimate_brokerage():
    return BROKERAGE_PER_TRADE

def print_trade_summary(brokerage_log):
    total_brokerage = sum(entry['fees'] for entry in brokerage_log)
    print(f"Total Brokerage Paid: {total_brokerage:.2f}")
    print("Brokerage Log:", brokerage_log)

def select_strike_atm_or_otm(df, opt_type, premium_target, spot_price, tolerance=0.10):
    df = df.copy()
    df['mark_price'] = df['mark_price'].astype(float)
    df['strike_price'] = df['strike_price'].astype(float)

    min_premium = premium_target * (1 - tolerance)
    max_premium = premium_target * (1 + tolerance)

    filtered = df[
        (df['contract_type'] == opt_type) &
        (df['mark_price'] >= min_premium) &
        (df['mark_price'] <= max_premium)
    ]

    if opt_type == 'call_options':
        filtered = filtered[filtered['strike_price'] >= spot_price]
    else:
        filtered = filtered[filtered['strike_price'] <= spot_price]

    if not filtered.empty:
        filtered['spot_distance'] = (filtered['strike_price'] - spot_price).abs()
        return filtered.sort_values('spot_distance').iloc[:1]

    return pd.DataFrame()

def management_loop(
    df_options, entry_call_price, entry_put_price,
    call_prod_id, put_prod_id, lots, call_order_id, put_order_id, nearest_expiry
):
    current_call_prod_id = call_prod_id
    current_put_prod_id = put_prod_id
    call_entry_price = entry_call_price
    put_entry_price = entry_put_price

    straddle_reached = False
    iron_fly_active = False
    brokerage_log = []
    last_adjusted_leg = None  # informational only

    # Dynamic thresholds: always from last reentry
    last_call_reentry_price = call_entry_price
    last_put_reentry_price = put_entry_price

    print("=== Begin Dynamic Delta-Neutral Management (stepwise, strict ATM/OTM strikes only) ===")

    while not straddle_reached:
        spot_price = get_spot_price()

        live_tickers = get_live_option_tickers("BTC", nearest_expiry)
        df_tickers = pd.DataFrame(live_tickers)
        if 'strike_price' in df_tickers.columns:
            df_tickers['strike_price'] = df_tickers['strike_price'].astype(float)

        call_row = df_tickers[df_tickers['product_id'] == current_call_prod_id]
        put_row = df_tickers[df_tickers['product_id'] == current_put_prod_id]

        if call_row.empty or put_row.empty:
            print("Live price data not found (maybe contract expired/rolled).")
            break

        live_call_price = float(call_row.iloc[0]["mark_price"])
        live_put_price  = float(put_row.iloc[0]["mark_price"])
        call_strike     = float(call_row.iloc[0]["strike_price"])
        put_strike      = float(put_row.iloc[0]["strike_price"])

        print(f"\nLive Call [{current_call_prod_id}] Entry={call_entry_price:.2f} | Live={live_call_price:.2f} | Strike={call_strike}")
        print(f"Live Put  [{current_put_prod_id}] Entry={put_entry_price:.2f} | Live={live_put_price:.2f} | Strike={put_strike}")

        print(
            f"Adjustment check: Call = {live_call_price:.2f} >= {last_call_reentry_price:.2f}*{ADJUSTMENT_THRESHOLD} ? "
            f"{live_call_price >= last_call_reentry_price * ADJUSTMENT_THRESHOLD}"
        )
        print(
            f"Adjustment check: Put = {live_put_price:.2f} >= {last_put_reentry_price:.2f}*{ADJUSTMENT_THRESHOLD} ? "
            f"{live_put_price >= last_put_reentry_price * ADJUSTMENT_THRESHOLD}"
        )
        print("Last adjusted leg:", last_adjusted_leg)

        # PUT spike → exit & re-enter CALL (repeat allowed)
        if live_put_price >= last_put_reentry_price * ADJUSTMENT_THRESHOLD:
            print(
                f"Dynamic PUT adjustment: Put premium >= last put reentry * 1.3, "
                f"{live_put_price:.2f} >= {last_put_reentry_price:.2f}*{ADJUSTMENT_THRESHOLD}"
            )

            order_call_close = place_market_order(delta, current_call_prod_id, lots, 'buy')
            brokerage_log.append({'type': 'call-exit-dynamic', 'fees': estimate_brokerage()})

            live_tickers = get_live_option_tickers("BTC", nearest_expiry)
            df_tickers = pd.DataFrame(live_tickers)
            if 'strike_price' in df_tickers.columns:
                df_tickers['strike_price'] = df_tickers['strike_price'].astype(float)

            new_call_leg = select_strike_atm_or_otm(df_tickers, 'call_options', live_put_price, spot_price)
            if new_call_leg.empty:
                print("No suitable ATM/OTM CALL re-entry available at raised PUT premium; waiting.")
                print_trade_summary(brokerage_log)
                time.sleep(30)
                continue

            new_call_prod_id = int(new_call_leg['product_id'].values[0])
            order_call_open = place_market_order(delta, new_call_prod_id, lots, 'sell')
            brokerage_log.append({'type': 'call-reenter-dynamic', 'fees': estimate_brokerage()})

            current_call_prod_id = new_call_prod_id
            call_entry_price = float(order_call_open['average_fill_price'])

            # Update thresholds
            last_call_reentry_price = call_entry_price
            last_put_reentry_price = live_put_price
            last_adjusted_leg = 'put'

            print(f"NEW Dynamic Call Strike: {new_call_leg['strike_price'].values[0]}, Entry Premium: {call_entry_price:.2f}")

        # CALL spike → exit & re-enter PUT (repeat allowed)
        if live_call_price >= last_call_reentry_price * ADJUSTMENT_THRESHOLD:
            print(
                f"Call premium +30%! Exiting decayed PUT and opening new PUT at Call's raised premium "
                f"(nearest ATM/OTM). {live_call_price:.2f} >= {last_call_reentry_price:.2f}*{ADJUSTMENT_THRESHOLD}"
            )

            order_put_close = place_market_order(delta, current_put_prod_id, lots, 'buy')
            brokerage_log.append({'type': 'put-exit', 'fees': estimate_brokerage()})

            live_tickers = get_live_option_tickers("BTC", nearest_expiry)
            df_tickers = pd.DataFrame(live_tickers)
            if 'strike_price' in df_tickers.columns:
                df_tickers['strike_price'] = df_tickers['strike_price'].astype(float)

            new_put_leg = select_strike_atm_or_otm(df_tickers, 'put_options', live_call_price, spot_price)
            if new_put_leg.empty:
                print("No suitable ATM/OTM PUT re-entry; waiting.")
                print_trade_summary(brokerage_log)
                time.sleep(30)
                continue

            new_put_prod_id = int(new_put_leg['product_id'].values[0])
            order_put_open = place_market_order(delta, new_put_prod_id, lots, 'sell')
            brokerage_log.append({'type': 'put-reenter', 'fees': estimate_brokerage()})

            current_put_prod_id = new_put_prod_id
            put_entry_price = float(order_put_open['average_fill_price'])

            # Update thresholds
            last_put_reentry_price = put_entry_price
            last_call_reentry_price = live_call_price
            last_adjusted_leg = 'call'

            print(f"NEW Put Strike: {new_put_leg['strike_price'].values[0]}, Entry Premium: {put_entry_price:.2f}")

        # Iron fly convergence
        call_strike = float(
            df_tickers[df_tickers['product_id'] == current_call_prod_id]['strike_price'].values[0]
        )
        put_strike = float(
            df_tickers[df_tickers['product_id'] == current_put_prod_id]['strike_price'].values[0]
        )

        if call_strike == put_strike and not iron_fly_active:
            print("Strikes converged (straddle)! Initiating Iron Fly hedging.")
            straddle_reached = True

            live_tickers = get_live_option_tickers("BTC", nearest_expiry)
            df_tickers = pd.DataFrame(live_tickers)
            if 'strike_price' in df_tickers.columns:
                df_tickers['strike_price'] = df_tickers['strike_price'].astype(float)

            strike = call_strike

            otm_calls = df_tickers[
                (df_tickers['contract_type'] == 'call_options') &
                (df_tickers['strike_price'] > strike)
            ].sort_values('strike_price')

            if not otm_calls.empty:
                call_wing = otm_calls.iloc[0]
                order_call_wing = place_market_order(delta, int(call_wing['product_id']), lots, 'buy')
                brokerage_log.append({'type': 'wing-call', 'fees': estimate_brokerage()})
                print(f"Buying OTM call wing (strike {call_wing['strike_price']}): product_id {call_wing['product_id']}")
            else:
                print("No OTM call wing available to buy.")

            otm_puts = df_tickers[
                (df_tickers['contract_type'] == 'put_options') &
                (df_tickers['strike_price'] < strike)
            ].sort_values('strike_price', ascending=False)

            if not otm_puts.empty:
                put_wing = otm_puts.iloc[0]
                order_put_wing = place_market_order(delta, int(put_wing['product_id']), lots, 'buy')
                brokerage_log.append({'type': 'wing-put', 'fees': estimate_brokerage()})
                print(f"Buying OTM put wing (strike {put_wing['strike_price']}): product_id {put_wing['product_id']}")
            else:
                print("No OTM put wing available to buy.")

            iron_fly_active = True

        print_trade_summary(brokerage_log)
        time.sleep(30)

def main():
    products = get_products()
    df_products = pd.DataFrame(products)
    df_products['underlying_symbol'] = df_products['underlying_asset'].apply(
        lambda x: x.get('symbol') if isinstance(x, dict) else None
    )

    btc_options = df_products[
        (df_products['contract_type'].isin(['call_options', 'put_options'])) &
        (df_products['underlying_symbol'] == 'BTC')
    ].copy()

    btc_options['expiry_dt'] = pd.to_datetime(btc_options['settlement_time'])
    expiries = sorted(btc_options['expiry_dt'].unique())
    nearest_expiry_dt = expiries[0]
    nearest_expiry = pd.to_datetime(nearest_expiry_dt).strftime("%d-%m-%Y")

    print(f"Nearest expiry: {nearest_expiry}")

    live_tickers = get_live_option_tickers("BTC", nearest_expiry)
    df_tickers = pd.DataFrame(live_tickers)
    if 'strike_price' in df_tickers.columns:
        df_tickers['strike_price'] = df_tickers['strike_price'].astype(float)

    merged = btc_options[
        btc_options['expiry_dt'].dt.strftime("%d-%m-%Y") == nearest_expiry
    ].merge(
        df_tickers, left_on='id', right_on='product_id', suffixes=('', '_tick')
    ).copy()

    def safe_delta(x):
        if isinstance(x, dict) and "delta" in x:
            try:
                return float(x["delta"])
            except Exception:
                return None
        return None

    merged['delta'] = merged['greeks'].apply(safe_delta)
    merged['mark_price'] = merged['mark_price']

    call = merged[(merged['contract_type'] == 'call_options') & (merged['delta'].notnull())]
    put  = merged[(merged['contract_type'] == 'put_options') & (merged['delta'].notnull())]

    call_leg = call.iloc[(call['delta'] - DELTA_TARGET).abs().argsort()[:1]]
    put_leg  = put.iloc[(put['delta'] + DELTA_TARGET).abs().argsort()[:1]]

    print("CALL CONTRACT:")
    print(call_leg[['symbol', 'id', 'strike_price', 'expiry_dt', 'delta', 'mark_price']])
    print("PUT CONTRACT:")
    print(put_leg[['symbol', 'id', 'strike_price', 'expiry_dt', 'delta', 'mark_price']])

    assets = delta.get_assets()
    available_usd = None
    for asset in assets:
        balance = delta.get_balances(asset['id'])
        if balance and (asset.get('symbol', '').upper() == 'USD'):
            available_usd = float(balance.get('available_balance'))
            break

    print("Available Balance USD:", available_usd)

    lots = 1
    confirm = input("Type YES to place live market sell orders on Delta Exchange (REAL FUNDS): ")
    if confirm != "YES":
        print("Trade aborted by user.")
        return

    order_call = place_market_order(delta, int(call_leg['id'].values[0]), lots, 'sell')
    order_put  = place_market_order(delta, int(put_leg['id'].values[0]), lots, 'sell')

    entry_call_price = float(order_call['average_fill_price'])
    entry_put_price  = float(order_put['average_fill_price'])

    call_prod_id = int(call_leg['id'].values[0])
    put_prod_id  = int(put_leg['id'].values[0])
    call_order_id = order_call['id']
    put_order_id  = order_put['id']

    management_loop(
        merged, entry_call_price, entry_put_price,
        call_prod_id, put_prod_id, lots, call_order_id, put_order_id, nearest_expiry
    )

if __name__ == "__main__":
    main()
